In [4]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Conexión a Postgres usando .env

In [5]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")

url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(url)

# test conexión
with engine.connect() as conn:
    print(conn.execute(text("SELECT 'Conexión OK ✅'")).scalar())

Conexión OK ✅


# Cargar tablas desde Postgres (staging → pandas)

In [6]:
import pandas as pd
from sqlalchemy import text

# Usamos el engine del notebook anterior.

def read_table(schema, table):
    return pd.read_sql(f"SELECT * FROM {schema}.{table};", con=engine)

customers = read_table("staging", "customers_stg")
orders = read_table("staging", "orders_stg")
order_items = read_table("staging", "order_items_stg")
products = read_table("staging", "products_stg")
sellers = read_table("staging", "sellers_stg")
payments = read_table("staging", "payments_stg")
reviews = read_table("staging", "reviews_stg")
geolocation = read_table("staging", "geolocation_stg")
translation = read_table("staging", "translation_stg")

print("Tablas staging cargadas ✅")

Tablas staging cargadas ✅


# Función para escribir en analytics

In [7]:
def to_analytics(df: pd.DataFrame, table_name: str):
    df.to_sql(
        table_name,
        con=engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )
    print(f"Tabla analytics.{table_name} cargada ✅ | shape={df.shape}")

## dim_product

In [8]:
dim_product = products.merge(
    translation,
    on="product_category_name",
    how="left"
)

dim_product["product_category_name_english"] = dim_product["product_category_name_english"].fillna("unknown")

# Selección de columnas “útiles”
dim_product = dim_product[
    [
        "product_id",
        "product_category_name",
        "product_category_name_english",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm",
        "product_photos_qty",
        "product_name_lenght",
        "product_description_lenght",
    ]
].copy()

dim_product.head()

,product_id,product_category_name,product_category_name_english,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_photos_qty,product_name_lenght,product_description_lenght
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,perfumery,225.0,16.0,10.0,14.0,1.0,40.0,287.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,art,1000.0,30.0,18.0,20.0,1.0,44.0,276.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,sports_leisure,154.0,18.0,9.0,15.0,1.0,46.0,250.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,baby,371.0,26.0,4.0,26.0,1.0,27.0,261.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,housewares,625.0,20.0,17.0,13.0,4.0,37.0,402.0


## dim_customer

In [9]:
dim_customer = customers[ 
    ["customer_id", "customer_unique_id", "customer_zip_code_prefix", "customer_city", "customer_state"] 
].copy() 
# opcional: renombrar para más claridad en analítica 
dim_customer.rename(columns={ 
    "customer_zip_code_prefix": "zip_code_prefix", 
    "customer_city": "city", 
    "customer_state": "state" }, 
    inplace=True) 

dim_customer.head()

,customer_id,customer_unique_id,zip_code_prefix,city,state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## dim_seller

In [10]:
dim_seller = sellers[
    ["seller_id", "seller_zip_code_prefix", "seller_city", "seller_state"]
].copy()

dim_seller.rename(columns={
    "seller_zip_code_prefix": "zip_code_prefix",
    "seller_city": "city",
    "seller_state": "state"
}, inplace=True)

dim_seller.head()

,seller_id,zip_code_prefix,city,state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## dim_date (desde Orders)

In [11]:
dates = pd.to_datetime(orders["order_purchase_timestamp"]).dt.date
dim_date = pd.DataFrame({"date": pd.Series(dates).dropna().unique()})
dim_date["date"] = pd.to_datetime(dim_date["date"])

dim_date["date_id"] = dim_date["date"].dt.strftime("%Y%m%d").astype(int)
dim_date["year"] = dim_date["date"].dt.year
dim_date["month"] = dim_date["date"].dt.month
dim_date["month_name"] = dim_date["date"].dt.month_name()
dim_date["day"] = dim_date["date"].dt.day
dim_date["day_of_week"] = dim_date["date"].dt.dayofweek  # 0=Lunes
dim_date["day_name"] = dim_date["date"].dt.day_name()
dim_date["quarter"] = dim_date["date"].dt.quarter
dim_date["is_weekend"] = dim_date["day_of_week"].isin([5, 6])

dim_date = dim_date.sort_values("date").reset_index(drop=True)

dim_date.head()

,date,date_id,year,month,month_name,day,day_of_week,day_name,quarter,is_weekend
0,2016-09-04,20160904,2016,9,September,4,6,Sunday,3,True
1,2016-09-05,20160905,2016,9,September,5,0,Monday,3,False
2,2016-09-13,20160913,2016,9,September,13,1,Tuesday,3,False
3,2016-09-15,20160915,2016,9,September,15,3,Thursday,3,False
4,2016-10-02,20161002,2016,10,October,2,6,Sunday,4,True


## dim_geolocation

In [12]:
# 1) Base desde staging
dim_geolocation = geolocation.copy()

# 2) Renombrar para consistencia
dim_geolocation.rename(columns={
    "geolocation_zip_code_prefix": "zip_code_prefix",
    "geolocation_lat": "lat",
    "geolocation_lng": "lng",
    "geolocation_city": "city",
    "geolocation_state": "state",
}, inplace=True)

# 3) Asegurar tipos
dim_geolocation["zip_code_prefix"] = dim_geolocation["zip_code_prefix"].astype("int64")

# 4) Validación
print("Shape:", dim_geolocation.shape)
print("zip_code_prefix duplicados:", dim_geolocation["zip_code_prefix"].duplicated().sum())
dim_geolocation.head()


Shape: (19015, 5)
zip_code_prefix duplicados: 0


,zip_code_prefix,lat,lng,city,state
0,1001,-23.550190,-46.634024,sao paulo,SP
1,1002,-23.548146,-46.634979,sao paulo,SP
2,1003,-23.548994,-46.635731,sao paulo,SP
3,1004,-23.549799,-46.634757,sao paulo,SP
4,1005,-23.549456,-46.636733,sao paulo,SP


# TABLA FACT

## preparar orders_fact

De orders nos quedamos con lo necesario y creamos date_id.

In [13]:
orders_fact = orders[[
    "order_id",
    "customer_id",
    "order_status",
    "order_purchase_timestamp"
]].copy()

orders_fact["order_purchase_timestamp"] = pd.to_datetime(
    orders_fact["order_purchase_timestamp"],
    errors="coerce"
)

# date_id = YYYYMMDD (clave para dim_date)
orders_fact["date_id"] = orders_fact["order_purchase_timestamp"].dt.strftime("%Y%m%d").astype("Int64")

orders_fact.head()


,order_id,customer_id,order_status,order_purchase_timestamp,date_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,20171002
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,20180724
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,20180808
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,20171118
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,20180213


## merge

muchos items pertenecen a un order
Eso es many_to_one

In [15]:
fact_order_items = order_items.merge(
    orders_fact[["order_id", "customer_id", "order_status", "date_id"]],
    on="order_id",
    how="left",
    validate="many_to_one"
)

fact_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,date_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,20170913
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,20170426
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,20180114
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,20180808
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,20170204


## crear métricas (medidas)

In [16]:
fact_order_items["price"] = pd.to_numeric(fact_order_items["price"], errors="coerce")
fact_order_items["freight_value"] = pd.to_numeric(fact_order_items["freight_value"], errors="coerce")

fact_order_items["item_total"] = fact_order_items["price"] + fact_order_items["freight_value"]


## chequeos de integridad

In [17]:
print("Shape:", fact_order_items.shape)

# clave única
print("Duplicados (order_id, order_item_id):",
      fact_order_items.duplicated(subset=["order_id", "order_item_id"]).sum())

# joins OK
print("Nulos en customer_id:", fact_order_items["customer_id"].isna().sum())
print("Nulos en date_id:", fact_order_items["date_id"].isna().sum())

Shape: (112650, 11)
Duplicados (order_id, order_item_id): 0
Nulos en customer_id: 0
Nulos en date_id: 0


## quedarnos solo con columnas finales

In [19]:
fact_order_items = fact_order_items[[
    "order_id",
    "order_item_id",
    "date_id",
    "customer_id",
    "product_id",
    "seller_id",
    "order_status",
    "price",
    "freight_value",
    "item_total",
    "shipping_limit_date"
]].copy()

fact_order_items["shipping_limit_date"] = pd.to_datetime(
    fact_order_items["shipping_limit_date"],
    errors="coerce"
)

fact_order_items.head()


,order_id,order_item_id,date_id,customer_id,product_id,seller_id,order_status,price,freight_value,item_total,shipping_limit_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,20170913,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,delivered,58.90,13.29,72.19,2017-09-19 09:45:35
1,00018f77f2f0320c557190d7a144bdd3,1,20170426,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,delivered,239.90,19.93,259.83,2017-05-03 11:05:13
2,000229ec398224ef6ca0657da4fc703e,1,20180114,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,delivered,199.00,17.87,216.87,2018-01-18 14:48:30
3,00024acbcdf0a6daa1e931b038114c75,1,20180808,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,delivered,12.99,12.79,25.78,2018-08-15 10:10:18
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,20170204,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,delivered,199.90,18.14,218.04,2017-02-13 13:57:51


# Cargar a Postgres

In [20]:
# DIMENSIONES
to_analytics(dim_product, "dim_product")
to_analytics(dim_customer, "dim_customer")
to_analytics(dim_seller, "dim_seller")
to_analytics(dim_geolocation, "dim_geolocation")
to_analytics(dim_date, "dim_date")

# HECHOS
to_analytics(fact_order_items, "fact_order_items")


Tabla analytics.dim_product cargada ✅ | shape=(32951, 10)
Tabla analytics.dim_customer cargada ✅ | shape=(99441, 5)
Tabla analytics.dim_seller cargada ✅ | shape=(3095, 4)
Tabla analytics.dim_geolocation cargada ✅ | shape=(19015, 5)
Tabla analytics.dim_date cargada ✅ | shape=(634, 10)
Tabla analytics.fact_order_items cargada ✅ | shape=(112650, 11)
